In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
import json
import pandas as pd
import time
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
from IPython.display import Markdown

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 01 Get API Keys

In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [5]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [6]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI 
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# completion llm  
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.0  
) 

retriever = vectorStore.as_retriever(search_type="similarity", k=3)
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
  """
    Always add a kind and friendly message according to the context at the beginning of the response.
    Take the information from context[0]["metadata"]
    Then follow this format
    Title: [Title of the Recipe]
    Description: [Short Description of the Recipe]
    Preparation Time: [Preparation Time]
    Cooking Time: [Cooking Time]
    Difficulty: [Difficulty Level]
    Serves: [Number of Servings]
    Diet Type: [Diet Type]
    Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
    Ingredients: [List of Ingredients]
    Instructions: [Step-by-step Cooking Instructions]

    If you don't have any general information, just respond with "I don't know!"

    {context}
    {{metadata}}
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "Give the 03 recipes with spaghetti"})
print(response)
response["answer"]
display(Markdown(response["answer"]))

{'input': 'Give the 03 recipes with spaghetti', 'context': [Document(id='43', metadata={'calories': '517', 'carbs': '65g', 'cook_time': '35 mins', 'diet_type': 'Dairy-free', 'difficulty': 'Easy', 'fat': '18g', 'fibre': '15g', 'id': '43', 'ingredients': '350gwholemeal spaghetti, 2 tbsprapeseed oil, 2aubergines(about 550g), each sliced into 4 lengthways, then cut into cubes, 4large garlic cloves,finely chopped, 3 tbsptomato purée, 1 tbspbalsamic vinegar, 250mlvegetable stock,made with 1 tbsp bouillon powder (vegan, if needed), 330gcherry tomatoes,halved, 30gpack of basil,chopped, 1 tbspcapers, 50gwalnut pieces,lightly toasted and chopped if large', 'instructions': 'Cook the spaghetti following pack instructions, then drain, reserving 150ml of the cooking water. Meanwhile, heat the oil in a large non-stick pan over a medium heat. Add the aubergine and garlic, stir well, then cover and cook for 8-10 mins, stirring a few times. Mix in the tomato purée and vinegar, and cook for 2-3 mins more

Hello! I'm happy to share some delicious spaghetti recipes with you. Here are three tasty options:

---

**Title:** Spaghetti with Aubergine, Tomatoes, Capers, and Walnuts  
**Description:** A fibre-rich, healthy, and filling lunch that's part of our Healthy Diet Plan.  
**Preparation Time:** 10 minutes  
**Cooking Time:** 20 minutes  
**Difficulty:** Easy  
**Serves:** 4  
**Diet Type:** Vegetarian  
**Nutrition:** calories 400 | fat 15g | protein 12g | fiber 8g  
**Ingredients:**  
- Spaghetti  
- Aubergine  
- Tomatoes  
- Capers  
- Walnuts  
- Olive oil  
- Garlic  
- Fresh parsley  
- Salt and pepper  

**Instructions:**  
1. Cook spaghetti according to package instructions.  
2. In a pan, heat olive oil and sauté garlic until fragrant.  
3. Add diced aubergine and cook until soft.  
4. Stir in chopped tomatoes, capers, and walnuts.  
5. Season with salt and pepper, and cook for another 5 minutes.  
6. Toss the cooked spaghetti with the sauce and garnish with fresh parsley.  

---

**Title:** Super-Speedy Spaghetti with Garlic, Parsley, and Bacon  
**Description:** A super-simple and super-speedy spaghetti supper, perfect for days when your fridge is bare.  
**Preparation Time:** 5 minutes  
**Cooking Time:** 15 minutes  
**Difficulty:** Easy  
**Serves:** 2  
**Diet Type:** Non-Vegetarian  
**Nutrition:** calories 550 | fat 25g | protein 20g | fiber 3g  
**Ingredients:**  
- Spaghetti  
- Bacon  
- Garlic  
- Fresh parsley  
- Olive oil  
- Salt and pepper  

**Instructions:**  
1. Cook spaghetti according to package instructions.  
2. In a pan, cook bacon until crispy, then remove and set aside.  
3. In the same pan, sauté minced garlic in olive oil.  
4. Add the cooked spaghetti and toss to coat.  
5. Stir in chopped parsley and crumbled bacon.  
6. Season with salt and pepper before serving.  

---

**Title:** Spaghetti with Nduja, Spring Greens, and Breadcrumb Topping  
**Description:** A simple and speedy dish, ideal for a midweek meal.  
**Preparation Time:** 10 minutes  
**Cooking Time:** 15 minutes  
**Difficulty:** Easy  
**Serves:** 4  
**Diet Type:** Non-Vegetarian  
**Nutrition:** calories 500 | fat 20g | protein 18g | fiber 5g  
**Ingredients:**  
- Spaghetti  
- Nduja  
- Spring greens (like spinach or kale)  
- Breadcrumbs  
- Olive oil  
- Garlic  
- Parmesan cheese (optional)  
- Salt and pepper  

**Instructions:**  
1. Cook spaghetti according to package instructions.  
2. In a pan, heat olive oil and sauté minced garlic.  
3. Add nduja and cook until melted.  
4. Stir in spring greens and cook until wilted.  
5. Toss the cooked spaghetti with the nduja mixture.  
6. Top with breadcrumbs and optional Parmesan cheese before serving.  

---

I hope you enjoy trying out these recipes! Happy cooking!